In [10]:
import pandas as pd

In [11]:
data = pd.read_csv("../data/BindingDB_BindingDB_Inhibition.tsv", sep="\t", nrows=20)

In [12]:
data.head()

,BindingDB Reactant_set_id,Ligand SMILES,Ligand InChI,Ligand InChI Key,BindingDB MonomerID,BindingDB Ligand Name,Target Name Assigned by Curator or DataSource,Target Source Organism According to Curator or DataSource,Ki (nM),IC50 (nM),...,UniProt (SwissProt) Recommended Name of Target Chain,UniProt (SwissProt) Entry Name of Target Chain,UniProt (SwissProt) Primary ID of Target Chain,UniProt (SwissProt) Secondary ID(s) of Target Chain,UniProt (SwissProt) Alternative ID(s) of Target Chain,UniProt (TrEMBL) Submitted Name of Target Chain,UniProt (TrEMBL) Entry Name of Target Chain,UniProt (TrEMBL) Primary ID of Target Chain,UniProt (TrEMBL) Secondary ID(s) of Target Chain,UniProt (TrEMBL) Alternative ID(s) of Target Chain
0,9825,COc1cc2c(Nc3ccc(Cl)cc3Cl)c(cnc2cc1OCCCN1CCN(C)...,InChI=1S/C25H27Cl2N5O2/c1-31-7-9-32(10-8-31)6-...,MVQUQGLRQPMJPU-UHFFFAOYSA-N,4521,3-quinolinecarbonitrile 3::4-Phenylamino-3-qui...,Proto-oncogene tyrosine-protein kinase Src,NaN,NaN,8.7,...,Proto-oncogene tyrosine-protein kinase Src,SRC_HUMAN,P12931,"E1P5V4,Q76P87,Q86VB9,Q9H5A8",NaN,NaN,NaN,NaN,NaN,NaN
1,9826,COc1cc2c(Nc3cc(OC)c(OC)c(OC)c3)c(cnc2cc1OCCCN1...,InChI=1S/C28H35N5O5/c1-32-8-10-33(11-9-32)7-6-...,VOUVVBUKNLBXGS-UHFFFAOYSA-N,6121,3-quinolinecarbonitrile 4::6-methoxy-7-[3-(4-m...,Proto-oncogene tyrosine-protein kinase Src,NaN,NaN,5.1,...,Proto-oncogene tyrosine-protein kinase Src,SRC_HUMAN,P12931,"E1P5V4,Q76P87,Q86VB9,Q9H5A8",NaN,NaN,NaN,NaN,NaN,NaN
2,9827,COCCOc1cc2ncc(C#N)c(Nc3cc(OC)c(Cl)cc3Cl)c2cc1O...,InChI=1S/C23H23Cl2N3O5/c1-29-4-6-32-21-8-15-18...,VXEOIAFZWQMOEP-UHFFFAOYSA-N,6122,"3-quinolinecarbonitrile 8::4-[(2,4-dichloro-5-...",Proto-oncogene tyrosine-protein kinase Src,NaN,NaN,2.8,...,Proto-oncogene tyrosine-protein kinase Src,SRC_HUMAN,P12931,"E1P5V4,Q76P87,Q86VB9,Q9H5A8",NaN,NaN,NaN,NaN,NaN,NaN
3,9828,COCCOc1cc2ncc(C#N)c(Nc3ccc(Cl)cc3Cl)c2cc1OCCOC,InChI=1S/C22H21Cl2N3O4/c1-28-5-7-30-20-10-16-1...,DTLASHUZTAPRBX-UHFFFAOYSA-N,6123,"3-quinolinecarbonitrile 9::4-[(2,4-dichlorophe...",Proto-oncogene tyrosine-protein kinase Src,NaN,NaN,12.0,...,Proto-oncogene tyrosine-protein kinase Src,SRC_HUMAN,P12931,"E1P5V4,Q76P87,Q86VB9,Q9H5A8",NaN,NaN,NaN,NaN,NaN,NaN
4,9829,COCCOc1cc2ncc(C#N)c(Nc3cc(OC)c(OC)c(OC)c3)c2cc...,InChI=1S/C25H29N3O7/c1-29-6-8-34-20-12-18-19(1...,BTBPPLGKNLXRGR-UHFFFAOYSA-N,6124,"3-quinolinecarbonitrile 10::6,7-bis(2-methoxye...",Proto-oncogene tyrosine-protein kinase Src,NaN,NaN,25.0,...,Proto-oncogene tyrosine-protein kinase Src,SRC_HUMAN,P12931,"E1P5V4,Q76P87,Q86VB9,Q9H5A8",NaN,NaN,NaN,NaN,NaN,NaN
